In [29]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_checkpoint = "output/sgd-distilgpt2/checkpoint-270000"

tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"
model = GPT2LMHeadModel.from_pretrained(model_checkpoint)
model.eval()

break_tokens = tokenizer.encode(tokenizer.eos_token)
MAX_LEN = model.config.n_ctx

data = (
    open("resources/sgd_0_1_simpletod/validation_sgd", "r")
    .read()
    .splitlines()[:4]
)
data = [d.split("<|action|>")[0].strip() for d in data]
inputs = tokenizer(data, return_tensors="pt", padding=True)


In [31]:
output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,
    max_length=MAX_LEN,
    pad_token_id=0,
)

In [34]:
def get_action(sent):
    if "<|action|>" not in sent:
        return []
    elif "<|belief|>" in sent:
        tmp = (
            sent.split("<|belief|>")[-1]
            .split("<|response|>")[0]
            .split("<|action|>")[-1]
            .strip()
        )
    elif "<|action|>" in sent:
        tmp = sent.split("<|response|>")[0].split("<|action|>")[-1].strip()
    else:
        return []
    tmp = tmp.strip(" .,")
    tmp = tmp.replace("<|endofaction|>", "")
    tmp = tmp.replace("<|endoftext|>", "")
    action = tmp.split(",")
    new_action = []
    for act in action:
        if act == "":
            continue
        act = act.strip(" .,")
        if act not in new_action:
            act = act.split()
            if len(act) == 3:
                new_action.append(tuple(act))
    return new_action


decoded_outputs = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
for output in decoded_outputs:
    print(sorted(get_action(output)))


[('bus', 'request', 'day'), ('bus', 'request', 'depart'), ('bus', 'request', 'travelers')]
[('bus', 'request', 'depart'), ('bus', 'request', 'dest')]
[('bus', 'request', 'depart'), ('bus', 'request', 'dest')]
[('bus', 'confirm', 'day'), ('bus', 'confirm', 'depart'), ('bus', 'confirm', 'dest'), ('bus', 'confirm', 'time'), ('bus', 'confirm', 'travelers')]
